In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
customers = pd.read_csv(r"C:\Users\henis\Downloads\Customers.csv")
products = pd.read_csv(r"C:\Users\henis\Downloads\Products.csv")
transactions = pd.read_csv(r"C:\Users\henis\Downloads\Transactions.csv")

In [66]:
#Aggregating transaction data for each customer

transaction_agg = transactions.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'count'],  # Total spend, Transaction count
    'TransactionDate': ['min', 'max'],
    "Quantity": ["sum"]
}).reset_index()

transaction_agg.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in transaction_agg.columns.values]

#multi-level column names
transaction_agg.columns = ['_'.join(col).strip() for col in transaction_agg.columns.values]
transaction_agg = transaction_agg.reset_index()

#transaction data and customer profile
customer_data = pd.merge(customers, transaction_agg, on='CustomerID', how="left")

#Featuring from both customer profiles
features = customer_data[['Age', 'Amount_sum', 'Amount_count']] #demographics and Transaction behavior

#Normalizing
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

KeyError: 'CustomerID'

In [54]:
#Initializing variables 
best_db_index = float('inf')
best_k = None
best_kmeans = None
best_labels = None

#values of k (from 2 to 10)
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_features)
    
    #Calculate the Davies-Bouldin Index (DB Index) and Silhouette Score
    db_index = davies_bouldin_score(scaled_features, kmeans.labels_)
    
    # Store the best model with the lowest DB Index
    if db_index < best_db_index:
        best_db_index = db_index
        best_k = k
        best_kmeans = kmeans
        best_labels = kmeans.labels_

#optimal number of clusters and DB Index
print(f"Optimal number of clusters: {best_k}")
print(f"Davies-Bouldin Index: {best_db_index}")

NameError: name 'scaled_features' is not defined

In [52]:
#additional clustering metrics
sil_score = silhouette_score(scaled_features, best_labels)
print(f"Silhouette Score: {sil_score}")

NameError: name 'scaled_features' is not defined

In [54]:
#feature space to 2D 
pca = PCA(n_components=2)
pca_features = pca.fit_transform(scaled_features)

# DataFrame for the PCA results and labels
pca_df = pd.DataFrame(pca_features, columns=['PCA1', 'PCA2'])
pca_df['Cluster'] = best_labels

# Visualizing the clusters on a scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='Cluster', data=pca_df, palette='Set2', s=100, alpha=0.7)
plt.title(f'Customer Segmentation (K={best_k})')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend(title='Cluster', loc='upper right')
plt.show()

NameError: name 'scaled_features' is not defined